<a href="https://colab.research.google.com/github/allenyeh929/generative_ai/blob/main/hw12.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [26]:
!pip install -U langchain langchain-community faiss-cpu openai

In [27]:
from langchain_community.vectorstores import FAISS
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_community.llms import HuggingFaceHub
from langchain.chains import RetrievalQA
from langchain.document_loaders import TextLoader
from langchain.text_splitter import CharacterTextSplitter
from getpass import getpass
import os

In [28]:
os.environ["HUGGINGFACEHUB_API_TOKEN"] = getpass("🔑 Enter your Hugging Face Token: ")

🔑 Enter your Hugging Face Token: ··········


In [43]:
with open("data.txt", "w") as f:
    f.write("""Rasterization is a technique in computer graphics that converts 3D models into 2D images by mapping vertices to screen space and filling in the pixels. It is commonly used in real-time applications like video games because of its speed.

Ray tracing, on the other hand, simulates the behavior of light to produce more realistic images. It traces rays from the camera through each pixel and calculates interactions with scene objects, including reflections, refractions, and shadows.

The graphics pipeline consists of stages like vertex processing, primitive assembly, rasterization, and fragment shading. Shaders are programs that run on the GPU to determine the appearance of objects in a scene.

OpenGL and DirectX are APIs used for rendering graphics in applications. They provide functions for managing the graphics pipeline and interacting with the GPU.

Bounding volume hierarchies (BVH) and grids are acceleration structures used in ray tracing to reduce the number of intersection tests, improving performance significantly.""")

loader = TextLoader("data.txt")
documents = loader.load()

In [44]:
text_splitter = CharacterTextSplitter(chunk_size=300, chunk_overlap=50)
docs = text_splitter.split_documents(documents)

In [31]:
embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
vectorstore = FAISS.from_documents(docs, embeddings)

In [48]:
from transformers import pipeline
from langchain.llms import HuggingFacePipeline

# 使用本地 transformers pipeline 作為 LLM
qa_pipeline = pipeline("text2text-generation", model="google/flan-t5-base", max_new_tokens=256)
llm = HuggingFacePipeline(pipeline=qa_pipeline)

retriever = vectorstore.as_retriever()

Device set to use cuda:0


In [49]:
from langchain.prompts import PromptTemplate

template = """
You are a computer graphics professor. Use the following context to answer the question at the end.

Context:
{context}

Question:
{question}

Answer:
"""

prompt = PromptTemplate(
    input_variables=["context", "question"],
    template=template,
)

qa = RetrievalQA.from_chain_type(
    llm=llm,
    retriever=retriever,
    chain_type="stuff",
    chain_type_kwargs={"prompt": prompt}
)


In [37]:
!pip install -q gradio


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.2/54.2 MB 18.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 323.3/323.3 kB 23.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 95.2/95.2 kB 9.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.5/11.5 MB 91.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.0/72.0 kB 6.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.4/62.4 kB 6.3 MB/s eta 0:00:00


In [50]:
import gradio as gr

def chatbot_interface(question):
    prompt = f"You are a computer graphics professor. Answer the following question using the provided knowledge: {question}"
    answer = qa.run(prompt)
    note = "\n\n🔍 Note: Answer is generated based on retrieved content from your uploaded materials."
    return f"🧑‍💻 Question: {question}\n🤖 Answer: {answer}{note}"

gr.Interface(
    fn=chatbot_interface,
    inputs=gr.Textbox(label="💬 Enter your computer graphics question"),
    outputs=gr.Textbox(label="🧠 Chatbot Response with Notes"),
    title="Computer Graphics Expert Chatbot"
).launch()

It looks like you are running Gradio on a hosted a Jupyter notebook. For the Gradio app to work, sharing must be enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://37635b34061995434a.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
